In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test_data = pd.read_csv("../input/titanic/test.csv", index_col="PassengerId")
train_data = pd.read_csv("../input/titanic/train.csv", index_col="PassengerId")

In [ ]:
train_data.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.isnull().sum()
#len(train[train.Survived==0])


In [ ]:
visual = (train_data.Age).hist(bins=80)
visual.set_xlabel("age")
visual.set_ylabel("frequency")


In [ ]:
import seaborn as sns

ax = sns.countplot(x="Sex", hue="Survived", data=train_data)

In [ ]:
train = train_data.drop(['Cabin'], axis=1)
test = test_data.drop(['Cabin'], axis=1)


In [ ]:
test.isnull().sum()

In [ ]:

train["Embarked"].fillna("S", inplace = True)
test["Embarked"].fillna("S", inplace = True)



In [ ]:
train.describe()
train.rename(columns={'Name': 'Title'}, inplace=True)
test.rename(columns={'Name': 'Title'}, inplace=True)

In [ ]:
new =train["Title"].str.split(".", n = 1, expand = True)[0] 
#print(new)

train["Title"]=new.str.split(", ", n = 1, expand = True)[1]

new =test["Title"].str.split(".", n = 1, expand = True)[0] 
#print(new)

test["Title"]=new.str.split(", ", n = 1, expand = True)[1]


  


In [ ]:
train['Title'] = train['Title'].replace(['Ms', 'Mlle'], 'Miss')
train['Title'] = train['Title'].replace(['Mme', 'the Countess', 'Lady', 'Dona'], 'Mrs')
train['Title'] = train['Title'].replace(['Dr', 'Major', 'Col', 'Sir', 'Rev', 'Jonkheer', 'Capt', 'Don'], 'Mr')

test['Title'] = test['Title'].replace(['Ms', 'Mlle'], 'Miss')
test['Title'] = test['Title'].replace(['Mme', 'Countess', 'Lady', 'Dona'], 'Mrs')
test['Title'] = test['Title'].replace(['Dr', 'Major', 'Col', 'Sir', 'Rev', 'Jonkheer', 'Capt', 'Don'], 'Mr')

In [ ]:


master_mean = train[train["Title"]=="Master"].Age.mean()
mr_mean = train[train["Title"]=="Mr"].Age.mean()
mrs_mean = train[train["Title"]=="Mrs"].Age.mean()
miss_mean = train[train["Title"]=="Miss"].Age.mean()


print(master_mean, mr_mean, mrs_mean, miss_mean)

In [ ]:
train.loc[   (train['Title']=="Master") & (np.isnan(train["Age"])) , "Age"] = master_mean
train.loc[   (train['Title']=="Mr") & (np.isnan(train["Age"])) , "Age"] = mr_mean
train.loc[   (train['Title']=="Miss") & (np.isnan(train["Age"])) , "Age"] = miss_mean
train.loc[   (train['Title']=="Mrs") & (np.isnan(train["Age"])) , "Age"] = mrs_mean

test.loc[   (test['Title']=="Master") & (np.isnan(test["Age"])) , "Age"] = master_mean
test.loc[   (test['Title']=="Mr") & (np.isnan(test["Age"])) , "Age"] = mr_mean
test.loc[   (test['Title']=="Miss") & (np.isnan(test["Age"])) , "Age"] = miss_mean
test.loc[   (test['Title']=="Mrs") & (np.isnan(test["Age"])) , "Age"] = mrs_mean

#train.Age.fillna(mr_mean, inplace= True)
#test.Age.fillna(mr_mean, inplace=True)


In [ ]:
train.groupby(['Title']).agg(['count']).Survived

In [ ]:
# convert male to 0, female to 1

train['Sex']=train['Sex'].map({'male': 0, 'female': 1}).astype(int)
test['Sex']=test['Sex'].map({'male': 0, 'female': 1}).astype(int)


In [ ]:

train.dtypes



In [ ]:
train.loc [ train['Age']< 16  , "Age"] = 0 #child
train.loc [ (train['Age']>=16) & (train['Age']<30)   , "Age"] = 1 # young
train.loc [ (train['Age']>=30) & (train['Age']<50)   , "Age"] = 2 # mid 
train.loc [ train['Age']>=50  , "Age"] = 3 #old

test.loc [ test['Age']<16  , "Age"] = 0 # child
test.loc [ (test['Age']>=16) & (test['Age']<30)   , "Age"] = 1 # young
test.loc [ (test['Age']>=30) & (test['Age']<50)   , "Age"] = 2 # mid 
test.loc [ test['Age']>=50  , "Age"] = 3 #olds





In [ ]:
train['Age'] = train['Age'].apply(np.int64) # convert floats to ints
#print(train.groupby(['Age']).agg(['count']).Survived)

test['Age'] = test['Age'].apply(np.int64) # convert floats to ints
print(test.groupby(['Age']).agg(['count']).Fare)





In [ ]:
train['Relatives'] =  train['SibSp'] + train['Parch']
test['Relatives'] =  test['SibSp'] + test['Parch']

In [ ]:
plt.figure(figsize=(20,10))
plt.xlabel('Fare')
plt.ylabel('Density')
sns.kdeplot(train['Fare'],shade=True, bw=2)

In [ ]:

train['Embarked'] = train['Embarked'].map({'S': 0, 'C':1, 'Q':2 }).astype(int)
test['Embarked'] = test['Embarked'].map({'S': 0, 'C':1, 'Q':2 }).astype(int)

In [ ]:
train['Title'] = train['Title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3}).astype(int) 
test['Title'] = test['Title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3}).astype(int) 

In [ ]:
train.groupby(['Relatives']).agg(['count']).Survived

In [ ]:

train['Relatives'] = train['Relatives'].apply(lambda x: 1 if (x==2 or x==1) else 0 if x==0 else 2 )
test['Relatives'] = test['Relatives'].apply(lambda x: 1 if (x==2 or x==1) else 0 if x==0 else 2 )

In [ ]:
columns = ['Pclass', 'Title', 'Sex', 'Age', 'Embarked', 'Relatives']

x_train=train[columns]
y_train = train.Survived
x_test = test[columns]

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(
  #   x, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
cross_valid_scores = {}

In [ ]:
model = SVC()
parameters_SVM = [{'kernel': ['rbf', 'linear'], 'gamma': [0.001,0.01, 0.1, 1],
                             'C': [0.1, 1, 10, 100, 1000]}]
model_SVM = GridSearchCV(
    model, 
    parameters_SVM, 
    cv=5,
    scoring='accuracy',
)

model_SVM.fit(x_train, y_train)

print('-----')
print(f'Best parameters {model_SVM.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + \
    f'{model_SVM.best_score_:.3f}'
)
cross_valid_scores['SVM'] = model_SVM.best_score_
print('-----')

best_svm = model.set_params(**model_SVM.best_params_)
best_svm.fit(x_train, y_train)
svm_preds = best_svm.predict(x_test)


In [ ]:
from sklearn import tree
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier


In [ ]:
parameters_RFC=[{'max_depth': [5, 10, 25],
             'n_estimators': [10, 50, 100],
             'min_samples_leaf': [1, 2, 4],
             'min_samples_split': [2, 5, 10]}]

model = RandomForestClassifier()

model_RFC = GridSearchCV(
    model, 
    parameters_RFC, 
    cv=5,
    scoring='accuracy',
)

model_RFC.fit(x_train, y_train)

print('-----')
print(f'Best parameters {model_RFC.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + \
    f'{model_RFC.best_score_:.3f}'
)
cross_valid_scores['RFC'] = model_RFC.best_score_
print('-----')

best_rfc = model.set_params(**model_RFC.best_params_)
best_rfc.fit(x_train, y_train)
rfc_preds = best_rfc.predict(x_test)

In [ ]:
import xgboost as xgb
parameters_XGB= {
    'max_depth': [3, 5, 10, 20], 
    'n_estimators': [10, 50, 100, 250],
    'learning_rate': [0.01, 0.05, 0.1]
}

model = xgb.XGBClassifier()

model_XGB = GridSearchCV(
    model, 
    parameters_XGB, 
    cv=5,
    scoring='accuracy',
)

model_XGB.fit(x_train, y_train)

print('-----')
print(f'Best parameters {model_XGB.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + \
    f'{model_XGB.best_score_:.3f}'
)
cross_valid_scores['XGB'] = model_XGB.best_score_
print('-----')



best_xgb = model.set_params(**model_XGB.best_params_)
best_xgb.fit(x_train, y_train)
xgb_preds = best_xgb.predict(x_test)


In [ ]:
cross_valid_scores

In [ ]:


values = [svm_preds, rfc_preds, xgb_preds]

average = list(map(sum, zip(*values))) # if sum is 3 or 0, all three algorithms decided on the same output
disagreement = average.count(1) + average.count(2)
disagreement # there is only 1 disagreement among 3 algorithms

final = [int(round((x/3),0)) for x in average ]




In [ ]:

test_data = pd.read_csv("../input/titanic/test.csv")

In [ ]:
submission = pd.DataFrame(
    {
        'PassengerId': test_data["PassengerId"], 
        'Survived': final,
    }
)
submission.to_csv(f"submission_merged.csv", index=False)